# Parcours d'un réseau de routeurs avec le protocole OSPF

Nous avons étudié dans l’année les protocoles de routage RIP (Routing Information Protocol) et OSPF (Open Shortest Path First) utilisés sur Internet.

Un réseau de routeurs peut être représenté par un graphe non-orienté et pondéré : les sommets sont des ordinateurs ou des routeurs et les arêtes sont pondérées par la métrique du protocole OSPF (liée à la vitesse de transmission de la liaison.)


<img src="Reseau_OSPF.png">

# Modélisation en Python d’un réseau de routeurs et étude de chemins (de routes) utilisant le protocole OSPF.


<img src="reseau.png">

On peut modéliser ce réseau par une matrice d'adjacence (de dimension 6).

In [32]:
import matplotlib.pyplot as plt
from graphviz import Graph
import random 
import os

TYPE_SORTIE_FICHIER_DOT = False
FORMAT_FICHIER_IMAGE = "png"

def genere_reseau_aleatoire(nb_routeurs):
    """
    Génère aléatoirement un réseau de routeurs
    :param nb_retours: le nombre de routeurs
    :return: une matrice d'adjacence représentant le réseau
    A l'intersection de la ligne lig et de la colonne col, un coefficient non nul indique que les routeurs (lig + 1) et
    (col + 1) sont reliés et que leur métrique de transmission est donné par le coefficient de la matrice
    """
    matrice_adjacence = [[0 for loop in range(nb_routeurs)] for loop in range(nb_routeurs)]
    for no_routeur in range(nb_routeurs):
        liste_voisins_possibles = [no for no in range(no_routeur)]
        if len(liste_voisins_possibles) > 0:
            voisins_choisis = random.sample(liste_voisins_possibles,random.randint(1,len(liste_voisins_possibles)))
            for no_routeur_voisin in voisins_choisis:
                    matrice_adjacence[no_routeur][no_routeur_voisin] = random.randint(1,10)
                    matrice_adjacence[no_routeur_voisin][no_routeur] = matrice_adjacence[no_routeur][no_routeur_voisin]
    return matrice_adjacence

matrice_adjacence = genere_reseau_aleatoire(6)
for ligne in matrice_adjacence:
    print(ligne)


[0, 3, 10, 0, 2, 10]
[3, 0, 7, 3, 4, 0]
[10, 7, 0, 0, 0, 0]
[0, 3, 0, 0, 4, 0]
[2, 4, 0, 4, 0, 0]
[10, 0, 0, 0, 0, 0]


In [50]:
def affiche_graphe(matrice_adjacence,routeur_depart=None,routeur_arrivee=None,ospf_max=None,rip_max=None):
    """
    :param graphe:
    :return:
    """
    graph = Graph(comment='RIP - OSPF')
    # Les sommets du graphe
    if routeur_depart:
        no_routeur_depart = int(routeur_depart.split("_")[1])
        no_routeur_arrivee = int(routeur_arrivee.split("_")[1])
        for lig in range(len(matrice_adjacence)):
            nom_routeur = "Routeur_" + str(lig + 1)
            if no_routeur_depart == lig + 1 or no_routeur_arrivee == lig + 1:
                graph.node(nom_routeur,color='green',style='filled')
    # les arêtes du graphe
    for lig in range(len(matrice_adjacence)):
        for col in range(lig):
            if matrice_adjacence[lig][col] != 0:
                nom_routeur1 = "Routeur_" + str(lig + 1)
                nom_routeur2 = "Routeur_" + str(col + 1)
                metrique = str(matrice_adjacence[lig][col])
                if ospf_max and nom_routeur1 in ospf_max[1] and nom_routeur2 in ospf_max[1] and abs(ospf_max[1].index(nom_routeur1) - ospf_max[1].index(nom_routeur2))==1:
                    graph.edge(nom_routeur1, nom_routeur2, label=metrique,color='green',size='5')
                    if rip_max and nom_routeur1 in rip_max and nom_routeur2 in rip_max and abs(rip_max.index(nom_routeur1) - rip_max.index(nom_routeur2)) == 1:
                        graph.edge(nom_routeur1, nom_routeur2, label=metrique, color='red', size='5')
                elif rip_max and nom_routeur1 in rip_max and nom_routeur2 in rip_max and abs(rip_max.index(nom_routeur1) - rip_max.index(nom_routeur2))==1:
                    graph.edge(nom_routeur1, nom_routeur2, label=metrique,color='red',size='5')
                else:
                    graph.edge(nom_routeur1, nom_routeur2,label=metrique)
    prefixe_nom_fichier = 'reseau_' + str(len(matrice_adjacence)) 
    with open(prefixe_nom_fichier + '.dot','w') as f:
        f.write(graph.source)
    print(graph.source)
    graph.format = FORMAT_FICHIER_IMAGE
    graph.render(prefixe_nom_fichier)
    nom_fichier = prefixe_nom_fichier + "."+ FORMAT_FICHIER_IMAGE
    #print(nom_fichier)
    display(HTML("<img src='" + nom_fichier + "'>"))

    
import os
from IPython.display import display, HTML
matrice_adjacence = genere_reseau_aleatoire(20)
for ligne in matrice_adjacence:
    print(ligne)
affiche_graphe(matrice_adjacence)    

[0, 8, 5, 0, 0, 10, 6, 0, 8, 7, 0, 8, 7, 0, 0, 6, 0, 0, 7, 10]
[8, 0, 8, 6, 1, 0, 0, 2, 3, 2, 2, 3, 0, 5, 10, 7, 1, 8, 6, 8]
[5, 8, 0, 0, 0, 4, 3, 8, 0, 2, 10, 0, 7, 3, 5, 6, 1, 0, 4, 10]
[0, 6, 0, 0, 0, 5, 5, 3, 0, 10, 0, 0, 6, 6, 0, 9, 0, 0, 3, 9]
[0, 1, 0, 0, 0, 7, 3, 2, 5, 2, 0, 6, 4, 0, 0, 7, 0, 0, 1, 1]
[10, 0, 4, 5, 7, 0, 10, 9, 2, 3, 0, 0, 8, 1, 0, 5, 2, 0, 10, 5]
[6, 0, 3, 5, 3, 10, 0, 0, 1, 5, 0, 4, 6, 8, 0, 9, 1, 0, 8, 4]
[0, 2, 8, 3, 2, 9, 0, 0, 8, 2, 0, 7, 9, 3, 0, 4, 9, 0, 8, 0]
[8, 3, 0, 0, 5, 2, 1, 8, 0, 10, 0, 0, 0, 0, 1, 4, 0, 0, 8, 0]
[7, 2, 2, 10, 2, 3, 5, 2, 10, 0, 0, 10, 2, 0, 0, 1, 0, 8, 4, 1]
[0, 2, 10, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 9, 5]
[8, 3, 0, 0, 6, 0, 4, 7, 0, 10, 2, 0, 4, 10, 0, 8, 8, 10, 3, 4]
[7, 0, 7, 6, 4, 8, 6, 9, 0, 2, 4, 4, 0, 9, 0, 5, 0, 2, 2, 3]
[0, 5, 3, 6, 0, 1, 8, 3, 0, 0, 0, 10, 9, 0, 0, 9, 6, 9, 10, 10]
[0, 10, 5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 3, 2, 9, 9]
[6, 7, 6, 9, 7, 5, 9, 4, 4, 1, 0, 8, 5, 9, 1, 0, 8, 7, 3, 5]
[0